1.
Давайте проанализируем данные опроса 4361 женщин из Ботсваны: botswana.tsv

О каждой из них мы знаем:

сколько детей она родила (признак ceb)

возраст (age)

длительность получения образования (educ)

религиозная принадлежность (religion)

идеальное, по её мнению, количество детей в семье (idlnchld)

была ли она когда-нибудь замужем (evermarr)

возраст первого замужества (agefm)

длительность получения образования мужем (heduc)

знает ли она о методах контрацепции (knowmeth)

использует ли она методы контрацепции (usemeth)

живёт ли она в городе (urban)

есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [5]:
import pandas as pd
import numpy as np

In [88]:
botswana = pd.read_csv('botswana.tsv', sep = '\t')
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [6]:
np.unique(botswana.religion)

array(['catholic', 'other', 'protestant', 'spirit'], dtype=object)

Answer = 4

2.
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [7]:
len(botswana.dropna())

1834

Answer = 1834

3.
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак $x_1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак

$$x_{2}=\left\{\begin{matrix}
1, x_{1} = "не применимо"\\ 
0, "иначе";
\end{matrix}\right.$$

заменить "не применимо" в $x_{1}$ на произвольную константу $c$, которая среди других значений $x_{1}$ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида

$$y=\beta_0 + \beta_1 x_1 + \beta_2 x_2$$,

на тех объектах, где $x_{1}$ было измерено, регрессионное уравнение примет вид

$$y=\beta_0 + \beta_1 x$$,

а там, где $x_{1}$ было "не применимо", получится

$$y=\beta_0 + \beta_1 c + \beta_2$$.

Выбор $c$ влияет только на значение и интерпретацию $\beta_2$, но не $\beta_1$.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.

Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице $X$ будет мультиколлинеарность.

Замените NaN в признаке agefm на $c_{agefm}=0$.

У объектов, где nevermarr = 1, замените NaN в признаке heduc на $c_{heduc_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [89]:
#Создайте признак nevermarr, равный единице там, где в agefm пропуски.
botswana['nevermarr'] = botswana['agefm'].isna().apply(lambda x: 1 if x else 0)

In [90]:
#Удалите признак evermarr 
botswana = botswana.drop('evermarr', axis = 1)

In [91]:
#Замените NaN в признаке agefm на  𝑐𝑎𝑔𝑒𝑓𝑚=0
botswana.agefm = botswana.agefm.fillna(0)

In [92]:
#У объектов, где nevermarr = 1, замените NaN в признаке heduc на  𝑐ℎ𝑒𝑑𝑢𝑐1=−1 
botswana['heduc'] = botswana.heduc.fillna(botswana['nevermarr'].apply(lambda x: -1 if x == 1 else np.nan))

In [93]:
sum(botswana.heduc.isna())

123

Answer = 123

4.
Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения ($c_{idlnchld}$=-1, $c_{heduc_2}$=-2 (значение -1 мы уже использовали), $c_{usemeth}$=-1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [95]:
botswana['idlnchld_noans'] = botswana['idlnchld'].isna().apply(lambda x: 1 if x else 0)
botswana['heduc_noans'] = botswana['heduc'].isna().apply(lambda x: 1 if x else 0)
botswana['usemeth_noans'] = botswana['usemeth'].isna().apply(lambda x: 1 if x else 0)

In [97]:
botswana.idlnchld = botswana.idlnchld.fillna(-1)
botswana.heduc = botswana.heduc.fillna(-2)
botswana.usemeth = botswana.usemeth.fillna(-1)
botswana.dropna(inplace = True)

In [102]:
print('Размер матрицы: ', len(botswana)*len(botswana.columns))

Размер матрицы:  78264


5.
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$? 

Округлите до трёх знаков после десятичной точки.

In [103]:
import statsmodels.formula.api as smf

In [104]:
botswana.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [106]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + radio + tv + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=botswana)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        16:31:15   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Answer: $R^2$ = 0.644

6.
Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

Answer = 3

7.
Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [107]:
import statsmodels.stats.api as sms

In [108]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [110]:
fitted = m1.fit(cov_type='HC1')

In [111]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


8.
Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [114]:
botswana.drop(['religion', 'radio', 'tv'], axis = 1, inplace = True)

In [116]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=botswana)

In [117]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


Answer = 0.4672

9.
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением $c_{usemeth }=-1$, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [119]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + '\
             'agefm + heduc + urban + electric + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans', data=botswana)

F, p, k1 = m2.fit().compare_f_test(m3.fit())
p

3.1552009480371243e-40

Answer = 40

10.
Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [120]:
fitted2 = m2.fit()
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        16:50:43   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [121]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [122]:
fitted2 = m2.fit(cov_type='HC1')
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        16:52:41   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.